In [1]:
# !/usr/bin/env python3

#cd ~/Desktop/snet/gits/lang-learn-repo/alex_tests

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import sys  
import subprocess
import pywt as pywt
import itertools

from sklearn import metrics
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA

from mpl_toolkits import mplot3d

In [2]:
#def PCAmapper(s,data):

#def getCleanGrammar(s):

#def getRules(s,word):

#def getPossibleNextWords: 

#def main:
#  for t in timeStream:
#    read data at time t
#    group data by clusters
#
#    for s in signalClusters:
#      map data into the pca normaliced space used to 'train' the 'model' (need to see how to do this)
#      symbolize based on the location in the pca space
#      get the index used in the grammar file to refer to this word
#      filter only words that correspond to an anomaly
#      readGrammarFile of s
#      process the file to have each word and its rules in a easy to use format (need to see how to do this)
#
#      #this depends on how the grammar works, and i will ask clarification of that with andres
#      for a in anomalousWord:
#        find it in the processedGrammarFile
#        if t = 0:
#          get all the disjoints for that word
#          initialize disjoints with position 0
#        else:
#          for d in disjoints:
#            check if current word would be at position p of the disjoint
#            if it is:
#              position = position+1
#            else:
#              position = 0
#            
#            if position == maxPosition:
#              issue warning

In [3]:
def ruleToPhraseSets(rules):
    words = []
    for r in rules:
        r = r.replace("(","")
        r = r.replace(")","")
        new = r.split(" & ")
        words = words + [new]
    words = list(words for words,_ in itertools.groupby(words))
    
    words2 = []
    for w in words:
        w2 = list(set(w))
        w2.sort()
        words2 = words2 + [w2]
    
    #print(words)
    #print(words2)
    return words2

In [4]:
#- es despues de (o que el otro lo tiene a la izquierda)
#+ es andes de (o que el otro lo tiene al a derecha)
def phrasesFromSets(desc,sets):
    size = len(desc[0])
    phrases = []
    for s in sets:
        if len(s)==1:
            if(s[0][-1]=='-'):
                temp = s[0]
                temp = temp.replace('-','')
                temp = temp.replace(desc[0],'',1)
                phrase = temp + desc[0]
            elif(s[0][-1]=='+'):
                temp = s[0]
                temp = temp.replace('+','')
                temp = temp.replace(desc[0],'',1)
                phrase = desc[0] + temp
            else:
                print("error, no sign")
        else:
            phrase = ""
            unjoined = []
            count = len(s)
            for p in s:
                if(p[-1]=='-'):
                    temp = p[:-1]
                    temp = temp.replace(desc[0],'',1)
                    sub = temp + desc[0]
                elif(p[-1]=='+'):
                    temp = p[:-1]
                    temp = temp.replace(desc[0],'',1)
                    sub = desc[0] + temp
                else:
                    print("error, no sign")
                    
                if(count<len(s)):
                    if (phrase[:size]==sub[-size:]):
                        phrase = sub + phrase[size:]
                    elif (sub[:size]==phrase[-size:]):
                        phrase = phrase[:size] + sub
                    else:
                        #-----------#revisar importancia el orden #--------------------------#
                        if(p[-1]=='-'):
                            phrase = sub + phrase[size:]
                            
                        elif(p[-1]=='+'):
                            phrase = phrase[:size] + sub
                            
                        #unjoined = unjoined + [sub]
                        #print(s,"error, no join")
                        #-----------#revisar importancia el orden #--------------------------#
                else:
                    phrase = sub
                    
                count = count-1
        phrases = phrases + [phrase]
    return phrases

In [5]:
def getCleanGrammar(s):  
    print(s)
    with open(s, 'r') as f:
        data = f.readlines()
 
    del data[0]
    del data[0]
    del data[0]
    del data[0]
    del data[-1]
    del data[-1]
    del data[-1]
    del data[-1]
    
    full = len(data)
    for r in range(full):
        line = data[full-r-1]
        if(len(line)==1):
            del data[full-r-1]
        elif(line[0]=="%"):
            data[full-r-1] = line[2:-1]
        elif(line[0]=="\""):
            data[full-r-1] = line[1:-3]
        else:
            data[full-r-1] = line[:-2].split(" or ")
    
    clean = []
    mapper = {}
    for r in range(0,len(data),3): 
        joined = [data[r],data[r+1]]
        mapper[joined[1]]=joined[0]
        phraseSets = ruleToPhraseSets(data[r+2])
        phrases = phrasesFromSets(joined,phraseSets)
        joined = joined + [phrases]
        clean = clean + [joined]
        
    return mapper,clean

In [6]:
def getBadRules(clean,maps,keys,exAnomaly,limit):
    size = len(clean[0][0])
    maps2 = {}
    for m in maps:
        maps2[maps[m]]=m
    
    
    clean2 = {}
    for c in clean:
        clean2[c[0]]=c[2]
    
    badRules = []
    badRuleWord = []
    badRuleCounter = []
    badRuleWordPos = []
    for rule in clean:
        if rule[1] in exAnomaly:
            badRules = badRules + rule[2]
            
    badRules = list(set(badRules))
    badRules.sort()
    badRuleWord = []
    badRuleCounter = []
    badRuleWordPos = []
       
    r=0
    stop = False
    newRules = badRules
    tester = 1
    while tester != 0:
        badRules = list(set(badRules + newRules))
        badRules.sort()
        newRules = []
        stop = True
        
        tester = len(badRules)
        for rule in badRules:
            if maps2[rule[:size]] in exAnomaly:
                #la primera palabra es anomalia, no hay que agregarle
                tester = tester-1
            elif len(rule)/size >= limit:
                #ya esta muy grande la oracion
                tester = tester-1
            else:
                subRules = []
                for c in clean2[rule[:size]]:
                    if rule[:size]==c[-size:]:
                        subRules = subRules + [c[:-size]+rule]
                
                if(len(subRules)>0):
                    badRules.remove(rule)
                    newRules = newRules + subRules
                else:
                    tester = tester-1
    
    for rule in badRules:
        pos = 999
        add = ""
        for anomaly in exAnomaly:
            loc = rule.find(maps[anomaly])
            if (loc >= 0 and loc < pos):
                pos = loc
                add = maps[anomaly]
        badRuleWord = badRuleWord + [add]       
        badRuleCounter = badRuleCounter + [0]
        badRuleWordPos = badRuleWordPos + [pos]
        r=r+1
        
    print(len(badRules))
    return badRules, badRuleWord, badRuleCounter, badRuleWordPos
    

In [7]:
limit=10
runPredictor=True

path = '/home/alexcasar/Desktop/snet/gits/lang-learn-repo/alex_tests/data'
filenames = ['data11']
filename = filenames[0]
doc = pd.read_csv(path+"/"+filename+"/"+filename+".csv") 

grouped = pd.read_csv(path+"/"+filename+"/"+filename+"_PCA_3_3.csv") 
grouped['anomaly']=list(doc['anomaly'])
groupedAno = grouped[grouped['anomaly']==1]
groupedAnoN = grouped[grouped['anomaly']==0]


signalClusters=['sym0','sym1','sym2','sym3','sym4','sym5','sym6']
for s in signalClusters:
    print(s)
    langFile = path+"/"+filename+"/lang/"+s+"_PCA_3_3/dict.txt"
    maps, clean = getCleanGrammar(langFile)
    keys = list(maps.keys())
    
    anomalies = list(groupedAno.groupby(by=s).count().iloc[:,0].index)
    nonanos = list(groupedAnoN.groupby(by=s).count().iloc[:,0].index)
    exAnomaly = []
    both = []
    for a in anomalies:
        if a not in nonanos:
            exAnomaly = exAnomaly + [a]
        else:
            both = both + [a]
    
    badRules, badRuleWord, badRuleCounter, badRuleWordPos = getBadRules(clean,maps,keys,exAnomaly,limit)
    
    if(runPredictor):
        print(len(grouped[s]))
        print(exAnomaly)
        a=0
        for word in grouped[s]:
            a=a+1
            if word in exAnomaly:
                if check:
                    badRuleCounter = [-1]*len(badRuleCounter)
                check = False
                print(a,word,'Anomaly NOW')
            else:  
                check = True
                r=0
                minW = 999
                pastword = badRules[0]
                for rule in badRules:
                    if maps[word] in rule:
                        if badRuleCounter[r] == -1 or pastword != word:
                            badRuleCounter[r] = rule.rfind(maps[word])-1
                        elif badRuleCounter[r] < badRuleWordPos[r]:
                            badRuleCounter[r] = badRuleCounter[r] + 1
                        pos = badRuleWordPos[r] - badRuleCounter[r]

                        if badRuleCounter[r] <= badRuleWordPos[r]:
                            minW = pos+1
                    #else:
                    #    badRuleCounter[r] = -1
                    pastword = word
                    r = r+1

                if minW < 999:# and minW > 0:
                    check = True
                    print(a,word,'Possible anomaly in',minW)
                else:
                    if check:
                        badRuleCounter = [-1]*len(badRuleCounter)
                    check = False
                    print(a,word,'Network is healthy')

  

sym0
/home/alexcasar/Desktop/snet/gits/lang-learn-repo/alex_tests/data/data11/lang/sym0_PCA_3_3/dict.txt
864
1153
['R14', 'R15', 'R16', 'R19', 'R24', 'R25', 'R26', 'R4']
1 R18 Possible anomaly in 8
2 R18 Possible anomaly in 7
3 R18 Possible anomaly in 6
4 R18 Possible anomaly in 5
5 R18 Possible anomaly in 4
6 R18 Possible anomaly in 3
7 R18 Possible anomaly in 2
8 R21 Possible anomaly in 1
9 R21 Possible anomaly in 1
10 R21 Possible anomaly in 1
11 R21 Possible anomaly in 1
12 R21 Possible anomaly in 1
13 R21 Possible anomaly in 1
14 R21 Possible anomaly in 1
15 R21 Possible anomaly in 1
16 R21 Possible anomaly in 1
17 R21 Possible anomaly in 1
18 R24 Anomaly NOW
19 R24 Anomaly NOW
20 R24 Anomaly NOW
21 R24 Anomaly NOW
22 R25 Anomaly NOW
23 R25 Anomaly NOW
24 R26 Anomaly NOW
25 R26 Anomaly NOW
26 R26 Anomaly NOW
27 R26 Anomaly NOW
28 R26 Anomaly NOW
29 R26 Anomaly NOW
30 R26 Anomaly NOW
31 R26 Anomaly NOW
32 R26 Anomaly NOW
33 R26 Anomaly NOW
34 R26 Anomaly NOW
35 R26 Anomaly NOW
36 R

623 R13 Possible anomaly in 1
624 R13 Possible anomaly in 1
625 R13 Possible anomaly in 1
626 R13 Possible anomaly in 1
627 R13 Possible anomaly in 1
628 R13 Possible anomaly in 1
629 R13 Possible anomaly in 1
630 R13 Possible anomaly in 1
631 R13 Possible anomaly in 1
632 R13 Possible anomaly in 1
633 R13 Possible anomaly in 1
634 R13 Possible anomaly in 1
635 R13 Possible anomaly in 1
636 R13 Possible anomaly in 1
637 R13 Possible anomaly in 1
638 R13 Possible anomaly in 1
639 R13 Possible anomaly in 1
640 R13 Possible anomaly in 1
641 R13 Possible anomaly in 1
642 R13 Possible anomaly in 1
643 R13 Possible anomaly in 1
644 R13 Possible anomaly in 1
645 R13 Possible anomaly in 1
646 R13 Possible anomaly in 1
647 R13 Possible anomaly in 1
648 R13 Possible anomaly in 1
649 R13 Possible anomaly in 1
650 R13 Possible anomaly in 1
651 R13 Possible anomaly in 1
652 R13 Possible anomaly in 1
653 R13 Possible anomaly in 1
654 R13 Possible anomaly in 1
655 R13 Possible anomaly in 1
656 R13 Po

1053 R14 Anomaly NOW
1054 R14 Anomaly NOW
1055 R14 Anomaly NOW
1056 R13 Possible anomaly in 3
1057 R13 Possible anomaly in 2
1058 R20 Possible anomaly in 3
1059 R20 Possible anomaly in 2
1060 R20 Possible anomaly in 1
1061 R19 Anomaly NOW
1062 R19 Anomaly NOW
1063 R10 Possible anomaly in 3
1064 R10 Possible anomaly in 2
1065 R10 Possible anomaly in 1
1066 R10 Possible anomaly in 1
1067 R10 Possible anomaly in 1
1068 R10 Possible anomaly in 1
1069 R10 Possible anomaly in 1
1070 R10 Possible anomaly in 1
1071 R10 Possible anomaly in 1
1072 R10 Possible anomaly in 1
1073 R10 Possible anomaly in 1
1074 R10 Possible anomaly in 1
1075 R10 Possible anomaly in 1
1076 R10 Possible anomaly in 1
1077 R10 Possible anomaly in 1
1078 R10 Possible anomaly in 1
1079 R10 Possible anomaly in 1
1080 R10 Possible anomaly in 1
1081 R10 Possible anomaly in 1
1082 R10 Possible anomaly in 1
1083 R10 Possible anomaly in 1
1084 R10 Possible anomaly in 1
1085 R10 Possible anomaly in 1
1086 R10 Possible anomaly i

642 R15 Possible anomaly in 1
643 R15 Possible anomaly in 1
644 R15 Possible anomaly in 1
645 R15 Possible anomaly in 1
646 R15 Possible anomaly in 1
647 R15 Possible anomaly in 1
648 R15 Possible anomaly in 1
649 R15 Possible anomaly in 1
650 R15 Possible anomaly in 1
651 R15 Possible anomaly in 1
652 R15 Possible anomaly in 1
653 R15 Possible anomaly in 1
654 R15 Possible anomaly in 1
655 R15 Possible anomaly in 1
656 R15 Possible anomaly in 1
657 R15 Possible anomaly in 1
658 R15 Possible anomaly in 1
659 R15 Possible anomaly in 1
660 R15 Possible anomaly in 1
661 R15 Possible anomaly in 1
662 R15 Possible anomaly in 1
663 R15 Possible anomaly in 1
664 R15 Possible anomaly in 1
665 R15 Possible anomaly in 1
666 R15 Possible anomaly in 1
667 R15 Possible anomaly in 1
668 R15 Possible anomaly in 1
669 R15 Possible anomaly in 1
670 R15 Possible anomaly in 1
671 R15 Possible anomaly in 1
672 R12 Anomaly NOW
673 R12 Anomaly NOW
674 R12 Anomaly NOW
675 R12 Anomaly NOW
676 R12 Anomaly NOW


193 R4 Anomaly NOW
194 R4 Anomaly NOW
195 R4 Anomaly NOW
196 R4 Anomaly NOW
197 R4 Anomaly NOW
198 R4 Anomaly NOW
199 R4 Anomaly NOW
200 R4 Anomaly NOW
201 R7 Anomaly NOW
202 R7 Anomaly NOW
203 R7 Anomaly NOW
204 R7 Anomaly NOW
205 R4 Anomaly NOW
206 R7 Anomaly NOW
207 R7 Anomaly NOW
208 R7 Anomaly NOW
209 R7 Anomaly NOW
210 R7 Anomaly NOW
211 R7 Anomaly NOW
212 R7 Anomaly NOW
213 R7 Anomaly NOW
214 R7 Anomaly NOW
215 R4 Anomaly NOW
216 R4 Anomaly NOW
217 R7 Anomaly NOW
218 R7 Anomaly NOW
219 R7 Anomaly NOW
220 R7 Anomaly NOW
221 R7 Anomaly NOW
222 R7 Anomaly NOW
223 R7 Anomaly NOW
224 R7 Anomaly NOW
225 R7 Anomaly NOW
226 R7 Anomaly NOW
227 R7 Anomaly NOW
228 R7 Anomaly NOW
229 R4 Anomaly NOW
230 R7 Anomaly NOW
231 R7 Anomaly NOW
232 R7 Anomaly NOW
233 R7 Anomaly NOW
234 R7 Anomaly NOW
235 R4 Anomaly NOW
236 R7 Anomaly NOW
237 R7 Anomaly NOW
238 R7 Anomaly NOW
239 R7 Anomaly NOW
240 R7 Anomaly NOW
241 R7 Anomaly NOW
242 R7 Anomaly NOW
243 R7 Anomaly NOW
244 R7 Anomaly NOW
245 R7 Anoma

...

413 R1 Network is healthy
414 R1 Network is healthy
415 R1 Network is healthy
416 R2 Possible anomaly in 1
417 R1 Network is healthy
418 R1 Network is healthy
419 R1 Network is healthy
420 R1 Network is healthy
421 R1 Network is healthy
422 R1 Network is healthy
423 R1 Network is healthy
424 R2 Possible anomaly in 1
425 R2 Possible anomaly in 1
426 R1 Network is healthy
427 R1 Network is healthy
428 R1 Network is healthy
429 R1 Network is healthy
430 R1 Network is healthy
431 R1 Network is healthy
432 R1 Network is healthy
433 R1 Network is healthy
434 R1 Network is healthy
435 R1 Network is healthy
436 R1 Network is healthy
437 R1 Network is healthy
438 R2 Possible anomaly in 1
439 R1 Network is healthy
440 R2 Possible anomaly in 1
441 R2 Possible anomaly in 1
442 R1 Network is healthy
443 R1 Network is healthy
444 R2 Possible anomaly in 1
445 R1 Network is healthy
446 R1 Network is healthy
447 R7 Possible anomaly in 7
448 R8 Possible anomaly in 6
449 R8 Possible anomaly in 5
450 R8 P

804 R26 Possible anomaly in 1
805 R26 Possible anomaly in 1
806 R24 Possible anomaly in 1
807 R13 Anomaly NOW
808 R22 Anomaly NOW
809 R22 Anomaly NOW
810 R22 Anomaly NOW
811 R22 Anomaly NOW
812 R21 Possible anomaly in 3
813 R22 Anomaly NOW
814 R22 Anomaly NOW
815 R22 Anomaly NOW
816 R22 Anomaly NOW
817 R14 Anomaly NOW
818 R22 Anomaly NOW
819 R22 Anomaly NOW
820 R14 Anomaly NOW
821 R14 Anomaly NOW
822 R22 Anomaly NOW
823 R22 Anomaly NOW
824 R14 Anomaly NOW
825 R14 Anomaly NOW
826 R13 Anomaly NOW
827 R14 Anomaly NOW
828 R12 Anomaly NOW
829 R14 Anomaly NOW
830 R14 Anomaly NOW
831 R14 Anomaly NOW
832 R22 Anomaly NOW
833 R13 Anomaly NOW
834 R13 Anomaly NOW
835 R13 Anomaly NOW
836 R13 Anomaly NOW
837 R13 Anomaly NOW
838 R14 Anomaly NOW
839 R14 Anomaly NOW
840 R14 Anomaly NOW
841 R14 Anomaly NOW
842 R13 Anomaly NOW
843 R12 Anomaly NOW
844 R12 Anomaly NOW
845 R14 Anomaly NOW
846 R12 Anomaly NOW
847 R12 Anomaly NOW
848 R14 Anomaly NOW
849 R12 Anomaly NOW
850 R12 Anomaly NOW
851 R14 Anomaly NOW


1178
1153
['R16', 'R18', 'R19', 'R20', 'R21', 'R22', 'R23', 'R24', 'R25', 'R26']
1 R3 Possible anomaly in 16
2 R3 Possible anomaly in 15
3 R3 Possible anomaly in 14
4 R3 Possible anomaly in 13
5 R3 Possible anomaly in 12
6 R3 Possible anomaly in 11
7 R3 Possible anomaly in 10
8 R14 Possible anomaly in 16
9 R3 Possible anomaly in 8
10 R14 Possible anomaly in 15
11 R3 Possible anomaly in 6
12 R13 Possible anomaly in 14
13 R3 Possible anomaly in 5
14 R12 Network is healthy
15 R12 Network is healthy
16 R8 Possible anomaly in 7
17 R8 Possible anomaly in 6
18 R6 Possible anomaly in 14
19 R6 Possible anomaly in 13
20 R4 Possible anomaly in 12
21 R8 Possible anomaly in 5
22 R6 Possible anomaly in 11
23 R6 Possible anomaly in 10
24 R6 Possible anomaly in 9
25 R6 Possible anomaly in 8
26 R6 Possible anomaly in 7
27 R7 Possible anomaly in 10
28 R8 Possible anomaly in 4
29 R6 Possible anomaly in 5
30 R8 Possible anomaly in 3
31 R4 Possible anomaly in 8
32 R4 Possible anomaly in 7
33 R4 Possible an

517 R14 Possible anomaly in 1
518 R14 Possible anomaly in 1
519 R10 Possible anomaly in 1
520 R3 Possible anomaly in 1
521 R3 Possible anomaly in 1
522 R4 Possible anomaly in 1
523 R5 Possible anomaly in 1
524 R3 Possible anomaly in 1
525 R4 Possible anomaly in 1
526 R3 Possible anomaly in 1
527 R5 Possible anomaly in 1
528 R3 Possible anomaly in 1
529 R4 Possible anomaly in 1
530 R3 Possible anomaly in 1
531 R4 Possible anomaly in 1
532 R3 Possible anomaly in 1
533 R3 Possible anomaly in 1
534 R3 Possible anomaly in 1
535 R3 Possible anomaly in 1
536 R3 Possible anomaly in 1
537 R5 Possible anomaly in 1
538 R5 Possible anomaly in 1
539 R3 Possible anomaly in 1
540 R5 Possible anomaly in 1
541 R3 Possible anomaly in 1
542 R3 Possible anomaly in 1
543 R3 Possible anomaly in 1
544 R5 Possible anomaly in 1
545 R3 Possible anomaly in 1
546 R5 Possible anomaly in 1
547 R5 Possible anomaly in 1
548 R5 Possible anomaly in 1
549 R3 Possible anomaly in 1
550 R4 Possible anomaly in 1
551 R4 Poss

969 R0 Possible anomaly in 11
970 R14 Possible anomaly in 11
971 R14 Possible anomaly in 10
972 R9 Possible anomaly in 9
973 R13 Possible anomaly in 2
974 R14 Possible anomaly in 8
975 R9 Possible anomaly in 7
976 R9 Possible anomaly in 6
977 R14 Possible anomaly in 5
978 R13 Possible anomaly in 1
979 R13 Possible anomaly in 1
980 R10 Possible anomaly in 4
981 R10 Possible anomaly in 3
982 R12 Network is healthy
983 R14 Possible anomaly in 12
984 R13 Possible anomaly in 5
985 R14 Possible anomaly in 11
986 R13 Possible anomaly in 3
987 R13 Possible anomaly in 2
988 R0 Possible anomaly in 10
989 R12 Network is healthy
990 R3 Possible anomaly in 8
991 R12 Network is healthy
992 R13 Possible anomaly in 8
993 R9 Possible anomaly in 8
994 R0 Possible anomaly in 5
995 R9 Possible anomaly in 7
996 R14 Possible anomaly in 6
997 R12 Network is healthy
998 R13 Possible anomaly in 8
999 R0 Possible anomaly in 8
1000 R13 Possible anomaly in 7
1001 R1 Possible anomaly in 8
1002 R14 Possible anomaly